In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs   
import time
import datetime
from tqdm import trange

In [2]:
# 스크래핑할 웹 사이트의 url을 선언
page_no = 1
url = "https://www.alba.co.kr/job/main?WsSrchKeywordWord=&hidschContainText=&hidWsearchInOut=&hidSort=&hidSortOrder=&hidSortDate=&hidListView=LIST&hidSortCnt=20&hidSortFilter=N&hidJobKind=&hidJobKindMulti=&page="+str(page_no)+"&hidSearchyn=Y&strAreaMulti=02%7C%7C%EA%B4%91%EC%A7%84%EA%B5%AC%7C%7C%EC%A0%84%EC%B2%B4&schtext=&hiretypecd=K01&lastschoolcd=&agelimit=&careercd=&searchterm=&hidCareerCD=&hidLastSchoolCD=&hidLastPayCD=&hidPayStart=&sortCnt=20"

In [3]:
# DataFrame 선언
#df = pd.DataFrame( columns = ["지역", "공고제목", "회사", "url"])
df = pd.DataFrame( columns = ["공고제목", "회사", "url"])


# 스크래핑할 웹 사이트의 총 페이지 수 파악
# '총 OOOO건' 이라는 검색 결과를 스크래핑하여 한 페이지당 표시 수인 20으로 나누기

response = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'})
soup = bs(response.text,'html.parser')

total_pages = int(soup.find('p', class_='listCount').find('strong').text.replace(',',''))
pages = total_pages // 20
print(total_pages, pages)

817 40


In [4]:
# For문을 이용해 웹 사이트 탐색
# 바깥의 for문을 통해 페이지를 바꾸어가며 탐색하고
# 안쪽의 for문으로 페이지 내 20개의 공고를 옮겨가며 탐색한다.

for i in trange(pages):
    url = "https://www.alba.co.kr/job/main?WsSrchKeywordWord=&hidschContainText=&hidWsearchInOut=&hidSort=&hidSortOrder=&hidSortDate=&hidListView=LIST&hidSortCnt=20&hidSortFilter=N&hidJobKind=&hidJobKindMulti=&page="+str(page_no)+"&hidSearchyn=Y&strAreaMulti=02%7C%7C%EA%B4%91%EC%A7%84%EA%B5%AC%7C%7C%EC%A0%84%EC%B2%B4&schtext=&hiretypecd=K01&lastschoolcd=&agelimit=&careercd=&searchterm=&hidCareerCD=&hidLastSchoolCD=&hidLastPayCD=&hidPayStart=&sortCnt=20"
    response = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'})
    soup = bs(response.text,'html.parser')

#     local = soup.find_all('td','local')
    title = soup.find_all('span','title')
    company = soup.find_all('span', 'company')
    each_url = soup.find_all('td', 'title')

    for j in range(20):
        df.loc[20*i + j] = [
        #local[j].text,
        title[j].text,
        company[j].text,
        "https://www.alba.co.kr"+each_url[j].find('a')['href']
        ]
    page_no += 1
    time.sleep(0.1)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:15<00:00,  2.65it/s]


In [5]:
file_name = str(datetime.date.today()) + "_알바천국_채용공고.csv"
df.to_csv(file_name, index=False, encoding='cp949')
pd.read_csv(file_name, encoding='cp949')

,공고제목,회사,url
0,샐러디 광장점 토/일 근무자 모집 (경력자 우대),샐러디 광장점,https://www.alba.co.kr/job/Detail?adid=1227450...
1,주말 알바 모집 합니다.,아차산점 아차산점,https://www.alba.co.kr/job/Detail?adid=1227396...
2,주말 알바 모집 합니다.,아차산점 아차산점,https://www.alba.co.kr/job/Detail?adid=1227396...
3,건대학생회관점(직영점)에서 함께 일할 스태프를 모집합니다.[주말 오전],CU 건대학생회관점,https://www.alba.co.kr/job/Detail?adid=1227243...
4,급구)CU뚝섬나들목점 토요일 오후 구합니다,CU 뚝섬나들목점,https://www.alba.co.kr/job/Detail?adid=1227448...
...,...,...,...
795,(+) 을지로입구역 / 하나은행 CS상담 [주말휴무/영업X/정착금지원],하나은행 고객센터(F&U),https://www.alba.co.kr/job/Detail?adid=1223775...
796,긴자료코 건대점에서 주방 정직원 모집합니다.,긴자료코 건대점,https://www.alba.co.kr/job/Detail?adid=1223554...
797,돼지한판 광정점 홀서빙 (직원 / 파트타임 ) 모집 합니다,최원석의돼지한판&서해쭈꾸미,https://www.alba.co.kr/job/Detail?adid=1223513...
798,[국민배달 배달대행] 광진구/성동구 라이더 모집,국민배달,https://www.alba.co.kr/job/Detail?adid=1221983...


In [6]:
url_list = list(df['url'])
print(len(url_list))

800


In [7]:
#######################################################

In [8]:
df2 = pd.DataFrame(columns = ["급여 종류", "급여 금액", "공고 등록일", "성별", "연령", "업직종", "근무기간", "근무요일", "근무시간", "지원마감일", "상세주소", "지원방법", "담당자이름", "담당자연락처"])
print(df2.shape)

(0, 14)


In [13]:
for i in range(len(url_list)):
    # 각 공고의 url 가져오기
    response = requests.get(url_list[i], headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'})
    soup = bs(response.text,'html.parser')
    
    ## 공고등록일
    reg_date_element = soup.find('div', 'detail-regist__date')
    if reg_date_element is not None:
        reg_date = reg_date_element.find('em').text
    else:
        df2.loc[i] = [None]*14
        continue
    
    # 모집 조건 리스트 선택(성별, 연령, 업직종(모집직종))
    condition_list = soup.find("div", class_="detail-content__condition-list")
    # 각 dl 태그에서 변수명과 변수값 가져오기
    details = {}
    dl_tags = condition_list.find_all("dl")
    for dl in dl_tags:
        dt = dl.find("dt").get_text(strip=True)
        if dl.find("dd")!=None:
            dd = dl.find("dd").get_text(strip=True)
        details[dt] = dd

    # 근무 조건 리스트 선택(급여, 근무기간, 근무요일, 근무시간)
    condition_list = soup.find("div", class_="detail-content__condition-list detail-content__list--last")
    # 각 dl 태그에서 변수명과 변수값 가져오기
    dl_tags = condition_list.find_all("dl")
    for dl in dl_tags:
        dt = dl.find("dt").get_text(strip=True)
        if dl.find("dd")!=None:
            dd = dl.find("dd").get_text(strip=True)
        if dt == "급여":
            pay_type = dl.find('i', 'detail-content__pay-icon').text
            pay_money = int(dl.find('strong').text.replace(',',''))
        details[dt] = dd
                
    # 지원방법
    apply = []
    apply_soup = soup.find('div', 'detail-content__recruit')
    a_tags = apply_soup.find_all('a')
    for a in a_tags:
        if a.find('i'):
            apply.append(a.find('i').text)
        elif "지원" in a.text:
            txt = a.text.replace('지원','').replace(' ','')
            if txt not in apply:
                apply.append(txt)
        elif "방문" in a.text:
            txt = a.text.replace(' 안내레이어 닫기', '')
            if txt not in apply:
                apply.append(txt)
    print(apply, url_list[i])
    
    # 공고마감일
    deadline = soup.find('strong', 'detail-content__recruit--date').text
        
    # 근무지주소
    condition_element = soup.find('div', 'InfoWork')
    dl_tags = soup.find_all('dl')
    address = ""
    for dl in dl_tags:
        dt = dl.find('dt')
        if dt.text == "근무지주소":
            dd = dl.find('dd')
            address = dd.text
            break
                    

    # 담당자이름 및 연락처
    manager_dls = soup.find('div', 'detail-content__recruit-list').find_all('dl')
    for dl in manager_dls:
        dt = dl.find('dt')
        if dt.text == "담당자명":
            dd = dl.find('dd')
            manager = dd.text
        if dt.text == "연락처":
            dd = dl.find('dd')
            manager_contact = dd.text
    df2.loc[i] = [pay_type, pay_money, reg_date, details["성별"], details["연령"], details["모집직종"], details["근무기간"], details["근무요일"], details["근무시간"], deadline, address, apply, manager, manager_contact]

['이메일', '전화후방문'] https://www.alba.co.kr/job/Detail?adid=122745084&listmenucd=ENTIRE


C:\ProgramData\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


['문자', '전화후방문'] https://www.alba.co.kr/job/Detail?adid=122739647&listmenucd=ENTIRE
['문자', '전화후방문'] https://www.alba.co.kr/job/Detail?adid=122739641&listmenucd=ENTIRE
['이메일', '전화후방문'] https://www.alba.co.kr/job/Detail?adid=122724314&listmenucd=ENTIRE
['전화후방문'] https://www.alba.co.kr/job/Detail?adid=122744882&listmenucd=ENTIRE
['전화후방문'] https://www.alba.co.kr/job/Detail?adid=122682010&listmenucd=ENTIRE
['문자', '전화후방문'] https://www.alba.co.kr/job/Detail?adid=122698749&listmenucd=ENTIRE
['문자', '전화후방문'] https://www.alba.co.kr/job/Detail?adid=122698718&listmenucd=ENTIRE


KeyboardInterrupt: 

In [10]:
df2.head()

,급여 종류,급여 금액,공고 등록일,성별,연령,업직종,근무기간,근무요일,근무시간,지원마감일,상세주소,지원방법,담당자이름,담당자연락처
0,시급,10000,2023.06.11(일),성별무관,연령무관,"외식·음료>일반음식점,패스트푸드서비스>매장관리·판매",6개월~1년 (협의가능),"토, 일요일 (협의가능, 주말 이외에 원하는 근무 요일 협의 가능)",17:00~21:00 (협의가능)(근무시간 및 요일 협의 가능하니 희망하는 요일 및...,2023.06.11(일) ~ 상시모집,서울 광진구 아차산로 475 1층 샐러디 광장점,"[이메일, 전화후방문]",유미애 부점장,\n\nEmail : mookcheeppa@naver.com\n
1,시급,10000,2023.06.10(토),성별무관,연령무관미성년자 불가,외식·음료>치킨·피자전문점서비스>캐셔·카운터,3개월~6개월,"금, 토, 일요일","금, 토, 일요일",2023.06.10(토) ~ 2023.06.16(금),서울 광진구 영화사로 40 (평강빌딩),"[문자, 전화후방문]",임준혁,\nH.P : 확인하기\nEmail : wnsgur820@naver.com\n
2,시급,10000,2023.06.10(토),성별무관,연령무관미성년자 불가,외식·음료>치킨·피자전문점서비스>캐셔·카운터,3개월~6개월,"금, 토, 일요일","금, 토, 일요일",2023.06.10(토) ~ 2023.06.16(금),서울 광진구 영화사로 40 (평강빌딩),"[문자, 전화후방문]",임준혁,\nH.P : 확인하기\nEmail : wnsgur820@naver.com\n
3,시급,9620,2023.06.09(금),성별무관,연령무관미성년자 불가,유통·판매>편의점,3개월~6개월 (협의가능),"토, 일요일",휴게시간30분,2023.06.09(금) ~ 2023.06.16(금),서울 광진구 능동로 120 (건국대학교)내 학생회관 1층,"[이메일, 전화후방문]",점장,\n\nEmail : yoori0207@bgf.co.kr\n
4,시급,9620,2023.06.11(일),성별무관,연령무관미성년자 불가,유통·판매>편의점,3개월~6개월,주1일,주1일,2023.06.11(일) ~ 2023.06.16(금),서울 광진구 능동로 18 (이튼타워리버3차) 104호,[전화후방문],신주희,\nH.P : 확인하기H.P : 확인하기\nEmail : sjh851800@nave...


In [11]:
result = pd.concat([df,df2],axis=1)
result

,공고제목,회사,url,급여 종류,급여 금액,공고 등록일,성별,연령,업직종,근무기간,근무요일,근무시간,지원마감일,상세주소,지원방법,담당자이름,담당자연락처
0,샐러디 광장점 토/일 근무자 모집 (경력자 우대),샐러디 광장점,https://www.alba.co.kr/job/Detail?adid=1227450...,시급,10000,2023.06.11(일),성별무관,연령무관,"외식·음료>일반음식점,패스트푸드서비스>매장관리·판매",6개월~1년 (협의가능),"토, 일요일 (협의가능, 주말 이외에 원하는 근무 요일 협의 가능)",17:00~21:00 (협의가능)(근무시간 및 요일 협의 가능하니 희망하는 요일 및...,2023.06.11(일) ~ 상시모집,서울 광진구 아차산로 475 1층 샐러디 광장점,"[이메일, 전화후방문]",유미애 부점장,\n\nEmail : mookcheeppa@naver.com\n
1,주말 알바 모집 합니다.,아차산점 아차산점,https://www.alba.co.kr/job/Detail?adid=1227396...,시급,10000,2023.06.10(토),성별무관,연령무관미성년자 불가,외식·음료>치킨·피자전문점서비스>캐셔·카운터,3개월~6개월,"금, 토, 일요일","금, 토, 일요일",2023.06.10(토) ~ 2023.06.16(금),서울 광진구 영화사로 40 (평강빌딩),"[문자, 전화후방문]",임준혁,\nH.P : 확인하기\nEmail : wnsgur820@naver.com\n
2,주말 알바 모집 합니다.,아차산점 아차산점,https://www.alba.co.kr/job/Detail?adid=1227396...,시급,10000,2023.06.10(토),성별무관,연령무관미성년자 불가,외식·음료>치킨·피자전문점서비스>캐셔·카운터,3개월~6개월,"금, 토, 일요일","금, 토, 일요일",2023.06.10(토) ~ 2023.06.16(금),서울 광진구 영화사로 40 (평강빌딩),"[문자, 전화후방문]",임준혁,\nH.P : 확인하기\nEmail : wnsgur820@naver.com\n
3,건대학생회관점(직영점)에서 함께 일할 스태프를 모집합니다.[주말 오전],CU 건대학생회관점,https://www.alba.co.kr/job/Detail?adid=1227243...,시급,9620,2023.06.09(금),성별무관,연령무관미성년자 불가,유통·판매>편의점,3개월~6개월 (협의가능),"토, 일요일",휴게시간30분,2023.06.09(금) ~ 2023.06.16(금),서울 광진구 능동로 120 (건국대학교)내 학생회관 1층,"[이메일, 전화후방문]",점장,\n\nEmail : yoori0207@bgf.co.kr\n
4,급구)CU뚝섬나들목점 토요일 오후 구합니다,CU 뚝섬나들목점,https://www.alba.co.kr/job/Detail?adid=1227448...,시급,9620,2023.06.11(일),성별무관,연령무관미성년자 불가,유통·판매>편의점,3개월~6개월,주1일,주1일,2023.06.11(일) ~ 2023.06.16(금),서울 광진구 능동로 18 (이튼타워리버3차) 104호,[전화후방문],신주희,\nH.P : 확인하기H.P : 확인하기\nEmail : sjh851800@nave...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,(+) 을지로입구역 / 하나은행 CS상담 [주말휴무/영업X/정착금지원],하나은행 고객센터(F&U),https://www.alba.co.kr/job/Detail?adid=1223775...,월급,2230000,2023.05.24(수),성별무관,연령무관,유통·판매>쇼핑몰·소셜커머스·홈쇼핑사무·회계>사무보조고객상담·영업·리서치>고객상담·...,1년이상 (협의가능),"월, 화, 수, 목, 금요일",휴게시간60분,2023.05.24(수) ~ 상시모집,서울 중구 다동 155 / 하나은행 고객센터,"[온라인, 이메일(자사양식), 간편입사, 홈페이지, 전화후방문]",김인숙,\nTEL : 확인하기\n\n
796,긴자료코 건대점에서 주방 정직원 모집합니다.,긴자료코 건대점,https://www.alba.co.kr/job/Detail?adid=1223554...,월급,3500000,2023.05.24(수),성별무관,연령무관,"외식·음료>일반음식점서비스>서빙,주방보조·설거지",6개월~1년,주6일,휴게시간90분,2023.05.24(수) ~ 상시모집,서울 광진구 능동로 133-1 .,"[문자, 전화후방문]",우현제,\nH.P : 확인하기\nEmail : predatore1@naver.com\n
797,돼지한판 광정점 홀서빙 (직원 / 파트타임 ) 모집 합니다,최원석의돼지한판&서해쭈꾸미,https://www.alba.co.kr/job/Detail?adid=1223513...,시급,13000,2023.05.24(수),성별무관,연령무관중장년 우대(만 40~65세),외식·음료>일반음식점서비스>서빙,6개월~1년 (수습기간있음),요일협의,시간협의(이미지 참조 바랍니다),2023.05.24(수) ~ 상시모집,서울 광진구 아차산로 478 (그레이스빌딩),"[문자, 전화후방문]",담당자,\nH.P : 확인하기H.P : 확인하기\nEmail : jjukkumi4560@h...
798,[국민배달 배달대행] 광진구/성동구 라이더 모집,국민배달,https://www.alba.co.kr/job/Detail?adid=1221983...,일급,200000,2023.05.24(수),성별무관,연령무관미성년자 불가,운전·배달>배달,3개월~6개월 (협의가능),주5일 (협의가능),로테이션시간협의,2023.05.24(수) ~ 상시모집,서울 광진구 답십리로 400-1 1층,[전화후방문],임광혁,\nH.P : 확인하기H.P : 확인하기\nEmail : gainaya@naver....


In [12]:
import codecs

file_name = str(datetime.date.today()) + "알바천국최종2.csv"
with codecs.open(file_name, 'w', encoding='cp949', errors='replace') as file:
    result.to_csv(file, index=False)
